# Qweb模板
- 是一个xml的模板引擎，用来生产html代码片段
- xml标签的属性指令需要t-作为前缀，使用格式如下：
        <t t-if="condition">
            <p>Test</p>
        </t>
- t标签是一个隐藏占位符，其显示的结果为：<p>Test</p>
- 如果是其他标签，t-if条件满足，则显示为：
        <div t-if="condition">  ==> <div>
    


## if判断条件
- 判断使用的关键字依然是if,elif,else，循环迭代使用的关键字为foreach
- 在qweb中使用foreach作为循环 遍历的关键字，使用t-as="xxx"作为迭代的子结果值的变量，然后可以使用t-esc="xxx"显示出所迭代的结果。例如：
        <t t-foreach="{1,2,3}" t-as="i">
            <p><t t-esc="i"/></p>
        </t>
- if和foreach都可以用于其他标签上
- foreach迭代之后传递给t-as的变量之后，在qweb这个变量还预置了下面几种属性
    - as_all:所有被迭代的对象
    - as_value:当前变量的值
    - as_index:当前变量在数组中的索引值
    - as_size:迭代数组的大小
    - as_first：是否为数组的第一个值
    - as_last：是否为数组的最后一个值
    - as_parity:当前迭代对象的奇偶
    - as_even：当前值在偶数索引上的布尔标志
    - as_odd:当前值在奇数索引上的布尔标志
              <t t-foreach="{1,2,3}" t-as="i">
                <p><t t-esc="i_index"/></p>
              </t>  
              
              <t t-foreach="{1,2,3}" t-as="i">
                <p><t t-esc="i_first"/></p>
              </t> 

# 更新或新增一个属性
- Qweb可以通过一些方法实现动态的属性计算，依赖t-att关键字，主要有下面几种形式
    - t-att-$name:创建一个名字为$name的属性，设置并展示属性的值
        - 设置属性
                 <div t-att-a="43"/>
        - 实际展示结果
                 <div a="42"></div>
    - t-attf-$name
        - 和t-att-$name类似，它不仅仅接受一个字符串，也可以是一个表达式
            - 设置属性
                    <t t-foreach="{1,2,3}" t-as="item">
                        <li t-attf-class="row{{item_parity}}"><t t-esc="item"/></li>
                    </t>
            - 通过循环遍历判断item的奇偶动态设置class属性的值
                - 实际展示结果
                        <li class="row even">1</li>
                        <li class="row odd">2</li>
                        <li class="row even">3</li>
    - t-att=mapping
        - 它的值也可以是一个key，value的映射形式
            - 设置属性
                    <div t-att="{'a':1, 'b':2}/>
            - 该属性的设置是以键为属性名，值为属性值
                - 实际展示结果
                        <div a="1" b="2"></div>
    - t-att=pair
        - 它的值格式为两个元素的数组：如下：
            - 设置属性
                     <div t-att="['a', 'b']"/>
            - 该属性是以数组中第一个元素为属性名，第二个元素为属性值
                - 实际展示结果
                        <div a="b"></div>

# 定义变量&sub-templates调用
## 1定义变量
- Qweb能够在在template中定义变量，这样更有利于清晰的表达一个值的表示含义
- 定义一个变量是通过self指令来完成的，而变量的值可以通过两种方式实现：
    - 一、通过t-value指定设置
        - <t t-set="foo" t-value="2 + 1"/>
        - <t t-sec="foo"/>
    - 输出内容：3
- 二、如果没有t-value指令设置值，则其子节点将被设置成变量的值
        <t t-set="foo">
            <li>ok</li>
        </t>
        <t t-esc="foo"/>
    - 输出内容：&li;li&gt;ok&lt;/li&gt;
    
## 2.sub-templates调用
- Qweb允许在模板中利用t-call指令调用子模块（sub-templates），用来避免重复命名，或这为模板的各个部分命名。首先我们定义了一个other-template的模板
        <!-- 定义other-template模板-->
            <template id="other-template">
                <p><t t-value="var"/></p>
            </template>
- 在另一个模板中使用t-call指定调用other-template模板
         <t t-call="other-template"/>
- 此时的渲染结果仅仅为一个p标签</p>,通过t-value给var变量设置值
         <t t-set="var" t-value="1"/>
         <t t-call="other-template"/>
- 这次的渲染结果为：<p>1</p>
- 不过上面的操作有一个问题是，我们是在t-call调整的var变量值（类似全局变量了).利用下面方式可以实现在t-call调用的字模板中可用：
        <t t-call="other-template">
            <t t-set="var" t-value="1"/>
        </t>
        <!-- "var" does not exist herr -->


# odoo中新建自己的js模块
- odoo中自定义js格式如下：
        odoo.define('module.Something', ['module.A', 'module.B'], function(require){
            "use strict"

            var A = require('module.A');
            var B = require('module.B');

            //some code

        })
    - odoo.define方法给了三个参数：
        - moduleName: javascript模块的名称。它应该是一个唯一的字符串。惯例是在odoo插件（addon）的名字后面加上一个具体的描述,也就是模块名加上当前js的用途描述。例如，“web.Widget”描述在web插件中定义的模块，该模块导出一个Widget类（因为第一个字母大写），如果名称不唯一，将引发异常并显示在控制台中
        - dependencies : 第二个参数是可选的，表示当前js的依赖，如果给定，它应该是一个字符串列表，每个字符串对应一个JavaScript模块。这描述了在执行模块之前需要加载的依赖项。如果这里没有明确地给出依赖项，那么模块系统将通过调用ToString从函数中提取它们，然后使用regexp查找所有Require语句。
        - 最后一个参数是一个函数function(),是定义模块的函数。它的返回值是模块的值，可以传递给其他需要它的模块。其中require是默认参数，js模块的返回值可以通过这个参数访问到
        - 注：require参数中包含了所有已经加载的js模块，这些模块是通过第一个参数进行访问的
    - 后台异步通信的代码格式如下:
        - 模块可能需要在准备就绪之前执行一些工作。例如，它可以做一个RPC来加载一些数据。在这种情况下，模块只需返回一个deferred（promise）。在这种情况下，模块系统只需等待deferred完成，然后注册模块。
                odoo.define('module.Something', ['web.ajax'], function (require) {
                    "use strict";

                    var ajax = require('web.ajax');

                    return ajax.rpc(...).then(function (result) {
                        // some code here
                        return something;
                    });
                });